In [38]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm
from sklearn.model_selection import GridSearchCV
import joblib
import logging

In [39]:

all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [40]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [42]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'


In [43]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [44]:

if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [45]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

In [46]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday']) 

In [48]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=180)
# principalComponents = pca.fit_transform(X)
# principalDf = pd.DataFrame(data = principalComponents)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

Random Forest Regressor

In [34]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(n_estimators = 225, n_jobs=-1, random_state=5)
model_rf.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.



RandomForestRegressor(n_estimators=225, n_jobs=-1, random_state=5)

Random Forest Evaluation

In [52]:
y_pred_train = model_rf.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = model_rf.predict(X_val)
y_pred_val[y_pred_val < 0] = 0



In [53]:
print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

RMS log-error train:  0.1534829529434426
RMS log-error val:  0.15549764095917318
RMS log-error train (actual):  0.4455391111780372
RMS log-error val (actual):  0.4544472954451142


In [53]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [55]:
y_pred_test = main_predict(model_rf, X_test)

[2.87898593 2.00142677 3.41190866 ... 1.71432635 1.84685877 3.40858016]
[0.55200858 0.43265692 2.39337453 ... 0.71178262 2.26748601 0.85403901]
[2.07503809 3.25801082 0.12817199 ... 2.50208397 2.8094404  3.16514565]
[0.69070445 0.063617   2.71970612 ... 1.24199547 1.03847298 3.13571799]
[2.27477229 0.87328752 1.15990637 ... 2.05039386 0.81654422 1.07981797]
[3.35334077 2.82755436 0.67315839 ... 0.47700917 2.74576169 1.71995474]
[2.39164095 2.93977061 3.06528515 ... 2.21278618 2.38289528 2.31924251]
[2.31524536 2.91144294 1.00380146 ... 1.2916191  0.36642512 1.6521123]
[1.86374573 3.28779725 0.46674664 ... 0.98148559 1.61864471 1.82164357]
[2.89191418 0.84684949 2.61592185 ... 0.24182667 3.29670147 3.34128918]
[2.82051876 1.60417511 0.36580434 ... 1.21699206 1.56942615 0.6283405]
[2.09959235 1.56566406 2.93132792 ... 0.86065851 3.01543831 2.66778904]
[1.18831648 1.70224429 2.32219878 ... 1.11162641 2.77446035 1.15720836]
[0.24770748 1.00555219 1.20337027 ... 0.03521269 1.91864885 0.6170

In [57]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submission_rf.csv', index=False)